In [98]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [2]:
def onoff_error(pred, gt, threshold):
    abs_error = np.abs(pred-gt)
    error = [x for x in abs_error.reshape(1, -1).tolist()[0] if x >= threshold]
    
    return np.mean(error)

In [3]:
# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]

In [157]:
threshold = {}
for appliance in ['dr', 'dw', 'mw']:
    sample_list = []
    for fold_num in range(5):
        sample_list = np.append(sample_list, [x for x in test_gt[fold_num][appliance].reshape(1, -1).tolist()[0] if x > 0])
    mean = np.mean(sample_list)
    print(appliance, mean)
    threshold[appliance] = 0.1*mean

dr 182.332933398
dw 117.045972072
mw 12.8771472261


In [158]:
threshold

{'dr': 18.233293339770761, 'dw': 11.704597207156935, 'mw': 1.2877147226060413}

In [23]:
gt = test_gt[0]['mw']
mean = np.mean([x for x in gt.reshape(1, -1).tolist()[0] if x > 0 ])
print(mean)

9.879183735


In [146]:
threshold = {"dr":10, "dw":10, "mw":5}

## Dataset 1: Artificial Aggregate Dataset

### Individual RNN and TreeRNN
#### Just use the best parameter found before as RNN always predict dryer as zero

In [246]:
dataset = 3
best_rnn_individual = np.load("../code/baseline/result/rnn-individual-param-{}.npy".format(dataset)).item()
best_rnn_tree = np.load("../code/baseline/result/rnn-tree-param-{}.npy".format(dataset)).item()

In [247]:
# get prediction for rnn individual
rnn_individual_pred = {}
rnn_individual_error = {}
for fold_num in range(5):
    rnn_individual_pred[fold_num] = {}
    rnn_individual_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        cell_type = best_rnn_individual[appliance][fold_num]['cell_type']
        hidden_size = best_rnn_individual[appliance][fold_num]['hidden_size']
        num_layers = best_rnn_individual[appliance][fold_num]['num_layers']
        lr = best_rnn_individual[appliance][fold_num]['lr']
        iters = best_rnn_individual[appliance][fold_num]['iters']
        bidirectional = best_rnn_individual[appliance][fold_num]['bidirectional']

        directory = "../code/baseline/rnn-tree/{}/{}/{}/{}/{}/{}/{}/3000/0.0/".format(dataset, fold_num, cell_type,
                                                                                                    hidden_size,
                                                                                                    num_layers,
                                                                                                    bidirectional,
                                                                                                    lr)
        filename = "test-pred-{}-{}-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, dataset, cell_type, hidden_size, num_layers,
                                                                                 bidirectional, lr, appliance)

        full_path = directory + filename
        rnn_individual_pred[fold_num][appliance] = np.asarray(np.load(full_path).item()[iters]).reshape(-1, 1, 112, 24)

In [248]:
# get prediction for rnn tree
rnn_tree_pred = {}
rnn_tree_error = {}
for fold_num in range(5):
    # get prediction for rnn tree
    rnn_tree_pred[fold_num] = {}
#     rnn_tree_error[fold_num] = {}
    cell_type = best_rnn_tree[fold_num]['cell_type']
    hidden_size = best_rnn_tree[fold_num]['hidden_size']
    num_layers = best_rnn_tree[fold_num]['num_layers']
    lr = best_rnn_tree[fold_num]['lr']
    iters = best_rnn_tree[fold_num]['iters']
    bidirectional = best_rnn_tree[fold_num]['bidirectional']
    order = best_rnn_tree[fold_num]['order']
    directory = "../code/baseline/rnn-tree/{}/{}/{}/{}/{}/{}/{}/3000/0.0/".format(dataset, fold_num, cell_type,
                                                                                                hidden_size,
                                                                                                num_layers,
                                                                                                bidirectional,
                                                                                                float(lr))
    filename = "test-pred-{}-{}-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, dataset, cell_type, hidden_size, num_layers,
                                                                             bidirectional, float(lr), str(order)[2:-2])

    full_path = directory + filename
    rnn_pred = np.asarray(np.load(full_path).item()[iters])
    

    for idx, appliance in enumerate(order):
        rnn_tree_pred[fold_num][appliance] = rnn_pred[idx].reshape(-1, 1, 112,24)

In [249]:
pd.DataFrame(calculate_error(rnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,1121.276868,317.134004,27.821228,263.290928,12.078453
1,1087.982078,323.810637,33.836725,338.311500,12.952859
2,1100.260138,302.197651,25.911348,304.010418,9.469221
3,595.084019,340.598606,30.200904,388.606593,7.225047
4,1119.461064,281.878602,30.280487,286.989624,17.829388


## Individual CNN and TreeCNN

In [203]:
valid_pred = {}
gid = 0
c = 0
for fold_num in range(5):
    valid_pred[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        valid_pred[fold_num][appliance] = {}
        for lr in [0.001, 0.01, 0.1]:
            valid_pred[fold_num][appliance][lr] = {}
            for iters in [200000]:
            
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "valid-pred-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                    c += 1
                else:
#                     continue
                    for it in range(1000, 200000+1, 1000):
                        valid_pred[fold_num][appliance][lr][it] = np.load(full_path).item()[it][0]
print(c)

0


In [176]:
valid_pred[fold_num][appliance][lr][it].reshape(-1, 24).shape

(1232, 24)

In [205]:
dataset

3

In [204]:
best_param = {}
for fold_num in range(5):
    best_param[fold_num] = {}
    for appliance in ['hvac', 'fridge']:
        best_param[fold_num][appliance] = {}
        min_error = np.inf
        for lr in [0.001, 0.01, 0.1]:
            for it in range(1000, 200000+1, 1000):
                error = mean_absolute_error(valid_pred[fold_num][appliance][lr][it].reshape(-1, 24), valid_gt[fold_num][appliance].reshape(-1, 24))
                if error < min_error:
                    best_param[fold_num][appliance]['lr'] = lr
                    best_param[fold_num][appliance]['iters'] = it
                    min_error = error

In [206]:
for fold_num in range(5):
    for appliance in ['dr', 'dw', 'mw']:
        best_param[fold_num][appliance] = {}
        min_error = np.inf
        for lr in [0.001, 0.01, 0.1]:
            for it in range(1000, 200000+1, 1000):
                error = onoff_error(valid_pred[fold_num][appliance][lr][it].reshape(-1, 24), valid_gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
                if error < min_error:
                    best_param[fold_num][appliance]['lr'] = lr
                    best_param[fold_num][appliance]['iters'] = it
                    min_error = error

In [207]:
np.save("../code/baseline/result/cnn-individual-new-param-{}.npy".format(dataset), best_param)

In [208]:
gid = 0
srv = 1
c = 0
cmd = {}
valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    valid_pred[fold_num] = {}
    for lr in [0.01]:
        valid_pred[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
            valid_pred[fold_num][lr][order] = {}


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-pred-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 1 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
                print(line)

                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                for it in range(1000, 20001, 1000):
                    valid_pred[fold_num][lr][order][it] = {}
                    for idx, appliance in enumerate(order):
                        valid_pred[fold_num][lr][order][it][appliance] = np.load(full_path).item()[it][idx]

print(c)                               

0


In [210]:
best_param = {}
for fold_num in range(5):
    best_param[fold_num] = {}
    min_error = np.inf
    for lr in [0.01]:
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
            for it in range(1000, 20001, 1000):
                error = 0
                for idx, appliance in enumerate(order):
                    if appliance in ['hvac', 'fridge']:
                        error += mean_absolute_error(valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                    valid_gt[fold_num][appliance].reshape(-1, 24))
                    else:
                        error += onoff_error(valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                    valid_gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
                if error < min_error:
                    min_error = error
                    best_param[fold_num]['lr'] = lr
                    best_param[fold_num]['order'] = order
                    best_param[fold_num]['iters'] = it
                
                    

In [211]:
np.save("../code/baseline/result/cnn-tree-new-param-{}.npy".format(dataset), best_param)

In [250]:
# load the best for cnn model
dataset = 3
best_cnn_individual = np.load("../code/baseline/result/cnn-individual-new-param-{}.npy".format(dataset)).item()
best_cnn_tree = np.load("../code/baseline/result/cnn-tree-new-param-{}.npy".format(dataset)).item()

In [251]:
# get prediction for rnn individual
cnn_individual_pred = {}
for fold_num in range(5):
    cnn_individual_pred[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        
        lr = best_cnn_individual[fold_num][appliance]['lr']
        iters = best_cnn_individual[fold_num][appliance]['iters']

        directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num,lr)
        filename = "test-pred-[\'{}\'].npy".format(appliance)

        full_path = directory + filename
        cnn_individual_pred[fold_num][appliance] = np.asarray(np.load(full_path).item()[iters]).reshape(-1, 1, 112, 24)

In [252]:
# get prediction for rnn individual
cnn_tree_pred = {}
for fold_num in range(5):
    cnn_tree_pred[fold_num] = {}
        
    lr = best_cnn_tree[fold_num]['lr']
    iters = best_cnn_tree[fold_num]['iters']
    order = best_cnn_tree[fold_num]['order']

    o = "\', \'".join(str(x) for x in order)
    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
    filename = "test-pred-[\'{}\'].npy".format(o)

    full_path = directory + filename
    cnn_pred = np.asarray(np.load(full_path).item()[iters])
    
    for idx, appliance in enumerate(order):
        cnn_tree_pred[fold_num][appliance] = cnn_pred[idx].reshape(-1, 1, 112,24)
    

In [253]:
def calculate_error(pred, gt, threshold):
    error = {}
    overall = {}
    num_homes = {}
    
    # calculte number of homes in each fold
    for fold_num in range(5):
        num_homes[fold_num] = gt[fold_num]['hvac'].reshape(-1, 1, 112,24).shape[0]
    homes = pd.Series(num_homes).sum()
    
    # for Constantly On appliances: hvac and fridge
    for appliance in ['hvac', 'fridge']:
        overall[appliance] = 0
        error[appliance] = {}
        for fold_num in range(5):
            error[appliance][fold_num] = mean_absolute_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                             gt[fold_num][appliance].reshape(-1, 24))
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
        
    # for ON/OFF appliances: dryer, dishwasher and microwave
    for appliance in ['dr', 'dw', 'mw']:
        error[appliance] = {}
        overall[appliance] = 0                                                                
        for fold_num in range(5):
            error[appliance][fold_num] = onoff_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                     gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
    
    
    return error, overall

In [254]:
new_error = {}
new_error['Individual RNN'] = calculate_error(rnn_individual_pred, test_gt, threshold)[1]
new_error['Tree RNN'] = calculate_error(rnn_tree_pred, test_gt, threshold)[1]
new_error['Individual CNN'] = calculate_error(cnn_individual_pred, test_gt, threshold)[1]
new_error['Tree CNN'] = calculate_error(cnn_tree_pred, test_gt, threshold)[1]

In [264]:
pd.DataFrame(calculate_error(cnn_individual_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,687.883952,314.129496,29.223454,301.734703,10.577555
1,789.291287,309.061034,34.489788,306.514438,13.688283
2,277.451695,297.465913,27.597581,299.928123,8.880822
3,526.526864,341.071813,32.830688,350.031428,7.231986
4,737.334451,281.647245,30.844946,304.567637,17.272667


In [265]:
pd.DataFrame(calculate_error(cnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,663.728323,230.762473,31.965210,293.099916,10.489638
1,766.104963,323.575743,38.008720,292.467895,14.129815
2,692.651233,302.196751,32.598922,302.566313,9.035520
3,472.208845,340.741854,33.941170,360.256777,8.047670
4,598.903175,281.877967,33.033803,318.398603,17.128898


In [255]:
pd.DataFrame(new_error)

,Individual CNN,Individual RNN,Tree CNN,Tree RNN
dr,602.867267,1010.684882,641.753523,1009.152254
dw,308.596146,313.261966,295.375694,313.179350
fridge,30.972570,28.942269,33.921979,29.591593
hvac,312.121611,324.961347,312.594083,315.607804
mw,11.509025,12.842696,11.742132,11.892869


In [187]:
pd.Series(calculate_error(rnn_individual_pred, test_gt, threshold)[1])

dr        1010.684882
dw         313.261966
fridge      28.942269
hvac       324.961347
mw          12.842696
dtype: float64

In [188]:
pd.DataFrame(calculate_error(rnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,1121.276868,317.134004,27.821228,263.290928,12.078453
1,1087.982078,323.810637,33.836725,338.311500,12.952859
2,1100.260138,302.197651,25.911348,304.010418,9.469221
3,595.084019,340.598606,30.200904,388.606593,7.225047
4,1119.461064,281.878602,30.280487,286.989624,17.829388


In [171]:
pd.DataFrame(calculate_error(cnn_individual_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,348.964577,317.134726,28.517904,175.616321,10.178631
1,492.447682,323.575744,34.600568,135.859949,13.226437
2,501.878728,302.197651,27.424491,160.653717,8.485112
3,346.853676,340.746708,32.630367,163.393771,7.281154
4,526.701238,281.879209,30.406227,151.497976,17.187027


In [172]:
pd.DataFrame(calculate_error(cnn_tree_pred, test_gt, threshold)[0])

,dr,dw,fridge,hvac,mw
0,231.665672,291.974351,30.586166,178.711373,10.915339
1,388.010323,319.754239,36.228537,133.157169,13.557611
2,384.176601,280.057231,35.868924,156.716972,8.913247
3,306.293603,323.724770,32.165805,166.607196,7.572600
4,429.115883,254.082933,32.605548,156.459055,16.713183
